In [ ]:
#| default_exp live_camera_feed

# Exploring Open CV

Import OpenCV

In [ ]:
#| export
import cv2


Let's check the connected cameras

In [ ]:
def get_available_cameras(limit=10):
    available_indices = []
    for i in range(limit):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_indices.append(i)
            cap.release()
    return available_indices

print(f"Connected camera indices: {get_available_cameras()}")

Connected camera indices: [0, 1]


We can provide an index to point to connected cameras (e.g., USB) or a URL for cameras connected wirelessly to the network

In [ ]:
#| export
camera_id = 0 # or an url for an IP camera "192.168.1.100:8080/video"
cap = cv2.VideoCapture(camera_id)


Some error handling and set-up of the sizing of the camera feed

In [ ]:
#| export
if not cap.isOpened():
    raise Exception("Could not open video device")

# Set frame size (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

True

Main loop to show the camera feed.

In [ ]:
#| export
print("Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Display the frame
    cv2.imshow('Live Camera', frame)
    
    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()